In [ ]:
import MeCab
mecab = MeCab.Tagger ('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import os
# import numpy as np

In [ ]:
DATAFOLDER = '../../../data_handson/matome/'
print(os.listdir(DATAFOLDER))

In [ ]:
path = DATAFOLDER + "matome_sample.html" # 'original_sample.html'
#  'http://alfalfalfa.com/articles/261820.html'
# 'http://tomcat.2ch.sc/test/read.cgi/livejupiter/1565300469/'

In [ ]:
with open(path) as f:
    text = f.read()
    
soup = bs(text, "html5lib")
res_list = soup.find_all("div", {'class':'res_block'})

res_num = [1]
for res in res_list[1:]:
    res_num.append(res.find('span',{'class':'res_num'}).text)

df_res_num = pd.DataFrame([res_num], index=['res_num']).T
df_res_num = df_res_num.reset_index(drop=False).rename(columns={'index':'matome_order'})
df_res_num['matome'] = 1
df_res_num['matome_order'] += 1
df_res_num['res_num'] = df_res_num['res_num'].astype('int')
df_res_num

In [ ]:
df_res_num.dtypes

In [ ]:
original_url = soup.find('div', {'class':'via'}).text[4:]
original_url

In [ ]:
def scrape_2ch(url):
    text = requests.get(original_url).text
    soup = bs(text, "html5lib")
    nums = []
    timestamps = []
    names = []
    idx = []
    bodies = []
    titles = []
    title = soup.find("title").text

    res_info_list = soup.find_all("dt")
    res_body_list = soup.find_all('dd')
    for info, body in zip(res_info_list, res_body_list):
        nums.append(int(info.text.split('：')[0]))
        names.append(info.text.split('：')[1])
        idx.append(info.text.split('：')[2].split(':')[-1])
        YYYY = info.text.split('：')[2].split('/')[0]
        MM =info.text.split('：')[2].split('/')[1]
        DD =info.text.split('：')[2].split('/')[2][:2]
        hh = info.text.split('：')[2].split(':')[0][-2:]
        mm = info.text.split('：')[2].split(':')[1]
        ss =  info.text.split('：')[2].split(':')[2][:2]
        timestamps.append(YYYY + '/' + MM + '/' + DD + ' ' + hh + ':' + mm + ':' + ss)
        bodies.append(body.text.split('\n')[0])
        titles.append(title)

    df = pd.DataFrame([nums, names, timestamps, idx, bodies, titles]).T
    df.columns = ['res_num', 'name', 'timestamp', 'idx', 'body', 'title']
    df['res_num'] = df['res_num'].astype('int')
    
    return df

In [ ]:
df_original = scrape_2ch(original_url)
df_original_flg = pd.merge(df_original, df_res_num, on='res_num',  how='left').fillna(0)

In [ ]:
df_original_flg['matome_order'] = df_original_flg['matome_order'].astype('int')
df_original_flg['matome'] = df_original_flg['matome'].astype('int')
df_original_flg